In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import SRW_v044 as SRW
from scipy.sparse import csr_matrix, csc_matrix, issparse
import functools

In [ ]:
nsamples = 100
nnodes = 1000
edge_freq = 0.004
cliq_edge_freq = 0.114
hi_mut_freq = 0.5
hi_node = nnodes*3/4
group_labels = ['Subtype 1']*(nsamples//2) + ['Subtype 2']*(nsamples//2)
feature_names = ['Subnetwork 1', 'Subnetwork 2', 'High mut source', 'High mut target', 
                 'Random 1', 'Random 2', 
                 'Self loop', 'Intercept']
node_names = ['{}'.format(i) for i in range(1,nnodes+1)]
sample_names = ['{}'.format(i) for i in range(1,nsamples+1)]

rand_mut_freq = 0.015

In [ ]:
rst_prob = 0.3
lam = 1e-1
WMW_b = 2e-4

In [ ]:
def accuracy(activation_func, n_iter = 10):
    accuracy_history = []
    accuracy_val_history = []
    while n_iter > 0:
        degrees = [0]*nnodes
        edges = []
        features = [] #(11) cliq1, cliq2, hi_mut_source, hi_mut_target, rand1, rand2, rand3, rand4, rand5, self_loop, intercept
        for i in range(nnodes-1):
            for j in range(i+1,nnodes):
                if ((i<100 and j<100) and np.random.random()<cliq_edge_freq) or np.random.random()<edge_freq:
                    edges.append([i,j])
                    edges.append([j,i])
                    features.append([0,0,0,0,np.random.random(),np.random.random(),0,1])
                    features.append([0,0,0,0,np.random.random(),np.random.random(),0,1])
                    if (i<50 and j<50):
                        features[-2][0] = 1
                        features[-1][0] = 1
                    if (i>=50 and i<100 and j>=50 and j<100):
                        features[-2][1] = 1
                        features[-1][1] = 1
                    if i == nnodes-1:
                        features[-2][2] = 1
                        features[-1][3] = 1
                    if j == nnodes-1:
                        features[-2][3] = 1
                        features[-1][2] = 1
                    degrees[i] += 1
                    degrees[j] += 1

        for i in range(nnodes):
            edges.append([i,i])
            features.append([0,0,0,0,np.random.random(),np.random.random(),1,1])

        P_init = []
        for p in range(nsamples):
            p_init = []
            for i in range(nnodes):
                freq=0
                if p == i:
                    freq = 1
                elif i == hi_node:
                    freq = hi_mut_freq
                elif i<100:
                    if (max(p,i)<50 or min(p,i)>=50):
                        freq = 0.015
                    else:
                        freq = 0.000
                else:
                    freq = rand_mut_freq

                if np.random.random() < freq:
                    p_init.append(1)
                else:
                    p_init.append(0)

            P_init.append(p_init)

        edges = np.array(edges)
        features = csc_matrix(features)
        P_init = csr_matrix(P_init)


        SRW_obj = SRW.SRW_solver(edges, features, nnodes, P_init, rst_prob, group_labels, lam, w_init_sd=0.01, 
                                 w=None, feature_names=feature_names, sample_names=sample_names, 
                                 node_names=node_names, loss='WMW', norm_type='L1', learning_rate=0.5, 
                                 update_w_func='Adam', P_init_val=P_init, group_labels_val=group_labels, 
                                 ncpus=len(feature_names), maxit=100, early_stop=10, WMW_b=WMW_b, activation_func = activation_func)

        SRW_obj.train_SRW_GD()
    
        accuracy_history.append(SRW_obj.accuracy)
        accuracy_val_history.append(SRW_obj.accuracy_val)
        n_iter -= 1
    return accuracy_history, accuracy_val_history

In [ ]:
accuracy_history_sigmoid, accuracy_val_history_sigmoid = accuracy('sigmoid')

In [ ]:
degrees = [0]*nnodes
edges = []
features = [] #(11) cliq1, cliq2, hi_mut_source, hi_mut_target, rand1, rand2, rand3, rand4, rand5, self_loop, intercept
for i in range(nnodes-1):
    for j in range(i+1,nnodes):
        if ((i<100 and j<100) and np.random.random()<cliq_edge_freq) or np.random.random()<edge_freq:
            edges.append([i,j])
            edges.append([j,i])
            features.append([0,0,0,0,np.random.random(),np.random.random(),0,1])
            features.append([0,0,0,0,np.random.random(),np.random.random(),0,1])
            if (i<50 and j<50):
                features[-2][0] = 1
                features[-1][0] = 1
            if (i>=50 and i<100 and j>=50 and j<100):
                features[-2][1] = 1
                features[-1][1] = 1
            if i == nnodes-1:
                features[-2][2] = 1
                features[-1][3] = 1
            if j == nnodes-1:
                features[-2][3] = 1
                features[-1][2] = 1
            degrees[i] += 1
            degrees[j] += 1

for i in range(nnodes):
    edges.append([i,i])
    features.append([0,0,0,0,np.random.random(),np.random.random(),1,1])

P_init = []
for p in range(nsamples):
    p_init = []
    for i in range(nnodes):
        freq=0
        if p == i:
            freq = 1
        elif i == hi_node:
            freq = hi_mut_freq
        elif i<100:
            if (max(p,i)<50 or min(p,i)>=50):
                freq = 0.015
            else:
                freq = 0.000
        else:
            freq = rand_mut_freq

        if np.random.random() < freq:
            p_init.append(1)
        else:
            p_init.append(0)

    P_init.append(p_init)

edges = np.array(edges)
features = csc_matrix(features)
P_init = csr_matrix(P_init)


SRW_obj = SRW.SRW_solver(edges, features, nnodes, P_init, rst_prob, group_labels, lam, w_init_sd=0.01, 
                         w=None, feature_names=feature_names, sample_names=sample_names, 
                         node_names=node_names, loss='WMW', norm_type='L1', learning_rate=0.5, 
                         update_w_func='Adam', P_init_val=P_init, group_labels_val=group_labels, 
                         ncpus=len(feature_names), maxit=100, early_stop=10, WMW_b=WMW_b, activation_func = 'sigmoid')

SRW_obj.train_SRW_GD()

In [ ]:
SRW_obj.w_map

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
SRW_obj.w = SRW_obj.w_list[0]
SRW_obj.map_w()
SRW_obj.w[-1] = 100
SRW_obj.w[-2] = -200
SRW_obj.Q = SRW.generate_Q(SRW_obj.edges, SRW_obj.nnodes, SRW_obj.features, SRW_obj.w)
P = SRW.iterative_PPR(SRW_obj.Q.toarray(), SRW.renorm(SRW_obj.P_init).toarray(), SRW_obj.rst_prob)
SRW_obj.C = SRW_obj.centroid(P, SRW_obj.ngroups, SRW_obj.group2indeces_list)
SRW_obj.calc_cost_and_acc_val()
SRW_obj.generate_Q_and_P_fin()

In [ ]:
P_df = SRW_obj.P_fin_df.copy()
pca = PCA(n_components=2)
pca.fit(P_df.T)
# P_df_pca, pca_components, explained_variance_ratio = pyNBS.run_pca(P_df)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
for i in range(len(group_labels)):
    if group_labels[i] == 'Subtype 1':
        ax.plot(pca.components_[0,i], pca.components_[1,i], alpha=0.5, marker='o', color = 'r', linestyle='')
    else:
        ax.plot(pca.components_[0,i], pca.components_[1,i], alpha=0.5, marker='o', color = 'b', linestyle='')

plt.xlabel('Component 1')
plt.ylabel('Component 2')
ax.legend(loc='upper right')

plt.show()

In [ ]:
SRW_obj.w = SRW_obj.w_list[-1]
SRW_obj.map_w()
SRW_obj.Q = SRW.generate_Q(SRW_obj.edges, SRW_obj.nnodes, SRW_obj.features, SRW_obj.w)
P = SRW.iterative_PPR(SRW_obj.Q.toarray(), SRW.renorm(SRW_obj.P_init).toarray(), SRW_obj.rst_prob)
SRW_obj.C = SRW_obj.centroid(P, SRW_obj.ngroups, SRW_obj.group2indeces_list)
SRW_obj.calc_cost_and_acc_val()
SRW_obj.generate_Q_and_P_fin()

P_df = SRW_obj.P_fin_df.copy()
pca = PCA(n_components=2)
pca.fit(P_df.T)

fig, ax = plt.subplots(figsize=(7, 7))
for i in range(len(group_labels)):
    if group_labels[i] == 'Subtype 1':
        ax.plot(pca.components_[0,i], pca.components_[1,i], alpha=0.5, marker='o', color = 'r', linestyle='')
    else:
        ax.plot(pca.components_[0,i], pca.components_[1,i], alpha=0.5, marker='o', color = 'b', linestyle='')

plt.xlabel('Component 1')
plt.ylabel('Component 2')
ax.legend(loc='upper right')

plt.show()

In [ ]:
P_df

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots(figsize=(32,18))
ax = sns.heatmap(P_df.clip(upper=0.025),cmap='Reds', square=False)
# plt.savefig('data/sim1001000_P_df.pdf')